In [1]:
import os
# change path to be in alpaca_rl_trader for linux
#os.chdir('/home/joe/Python/tradingbot/gym/alpaca_rl_trader')

# change path to be in alpaca_rl_trader for windows
os.chdir('C:/Users/JoeBa/Documents/python/alpaca_rl_trader_Discrete/')

In [2]:
from data.data_loader import load_data, SplitOption
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Get test data
data = load_data(
    ratio=0.8, split_option=SplitOption.NO_SPLIT, symbols=[], table_name='crypto_data_hourly')

c:\Users\JoeBa\Documents\python\virtual_environments\trading_env\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\JoeBa\Documents\python\virtual_environments\trading_env\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\JoeBa\Documents\python\virtual_environments\trading_env\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Trimming based on symbol(s): X:AVAXUSD
X:AAVEUSD: Trimmed 32.13%
X:MKRUSD: Trimmed 51.76%
X:LTCUSD: Trimmed 41.94%
X:ETHUSD: Trimmed 41.91%
X:CRVUSD: Trimmed 35.69%
X:BCHUSD: Trimmed 42.97%
X:LINKUSD: Trimmed 42.37%
X:BATUSD: Trimmed 61.30%
X:SHIBUSD: Trimmed 3.27%
X:BTCUSD: Trimmed 41.91%
X:GRTUSD: Trimmed 29.60%
X:XTZUSD: Trimmed 47.69%
X:UNIUSD: Trimmed 35.95%
X:DOTUSD: Trimmed 42.11%
X:AVAXUSD: Trimmed 0.00%


1. Data Preprocessing
The first step is to convert your DataFrame into a format that can be used for training the autoencoder. Each stock's data should be transformed into a sequence of fixed-length windows.

In [4]:
# Assuming you have 'price' and 'volume' columns
price_column = 'close'  # Replace with your actual price column name
volume_column = 'volume'  # Replace with your actual volume column name

# Function to create sequences with scaling applied to each sequence
def create_sequences(data):
    for i in range(len(data)):
        # Get the price and volume data for the current row
        close = np.log(data[price_column][i]+1)
        volume = np.log(data[volume_column][i]+1)
        
    return np.array([close, volume]) 

In [5]:
# Create sequences for each stock
stock_sequences = {}
for ticker in data.keys():
    stock_sequences[ticker] = create_sequences(data[ticker])

In [6]:
# Get the first ticker key from the sorted dictionary
first_ticker = next(iter(data))
length_of_data = len(data[first_ticker]['close'])

In [7]:
length_of_data

10030

In [8]:
stock_sequences[first_ticker]

array([4.97714745, 7.14622942])

2. Creating the Dataset for Training
Once you have the sequences, you can create a dataset that is suitable for training the autoencoder.

In [9]:
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch

# Convert sequences to a single NumPy ndarray
all_sequences = np.concatenate([np.array(sequences) for sequences in stock_sequences.values()], axis=0)

# Convert the NumPy ndarray to a PyTorch tensor
all_sequences = torch.tensor(all_sequences, dtype=torch.float32)

# Create DataLoader
batch_size = 64  # You can adjust this
dataset = TensorDataset(all_sequences, all_sequences)  # input and target are the same for autoencoders
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


3. Define the Autoencoder
Now, define the autoencoder model. You can use the example provided in the previous response or create your own architecture.

In [12]:
import torch.nn as nn

class TimeSeriesAutoencoder(nn.Module):
    def __init__(self):
        super(TimeSeriesAutoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(2, 64),  # Input size: 2 (price and volume), Output size: 64 (adjust as needed)
            nn.ReLU(),
            nn.Linear(64, 32),  # Output size: 32 (adjust as needed)
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),  # Input size: 32, Output size: 64 (mirroring the encoder)
            nn.ReLU(),
            nn.Linear(64, 2)  # Input size: 64, Output size: 2 (mirroring the input features)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Instantiate the autoencoder
autoencoder = TimeSeriesAutoencoder()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TimeSeriesAutoencoder(nn.Module):
    def __init__(self):
        super(TimeSeriesAutoencoder, self).__init__()
        # Encoder
        self.conv1 = nn.Conv1d(2, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv1d(32, 48, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2, 2)

        # Additional Pooling Layer
        self.additional_pool = nn.MaxPool1d(2, 2)

        # Bottleneck
        self.bottleneck = nn.Conv1d(48, 2, kernel_size=3, padding=1)  # Reduced channel dimensions

        # Decoder convolution layers
        self.deconv1 = nn.Conv1d(2, 48, kernel_size=3, padding=1)
        self.deconv2 = nn.Conv1d(48, 32, kernel_size=3, padding=1)
        self.deconv3 = nn.Conv1d(32, 16, kernel_size=3, padding=1)
        self.output_layer = nn.Conv1d(16, 2, kernel_size=3, padding=1)

    def forward(self, x):
        # Encode
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = self.additional_pool(x)
        encoded = self.bottleneck(x)  # Compressed representation

        # Decode
        x = F.interpolate(encoded, scale_factor=2, mode='nearest')
        x = F.relu(self.deconv1(x))
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = F.relu(self.deconv2(x))
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        x = F.relu(self.deconv3(x))
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        decoded = F.tanh(self.output_layer(x))

        return encoded, decoded


4. Train the Autoencoder
Finally, train the autoencoder. This involves passing your data through the model and using a loss function (like MSE) to update the weights.

In [13]:
# Define your autoencoder model
autoencoder = TimeSeriesAutoencoder()

# Define Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0001)

# Initialize the best loss value
best_loss = float('inf')

# Training Loop
num_epochs = 1000  # Number of epochs
for epoch in range(num_epochs):
    for data in data_loader:
        inputs, targets = data

        optimizer.zero_grad()
        encoded, decoded = autoencoder(inputs)
        loss = criterion(decoded, targets)
        loss.backward()
        optimizer.step()

    # Print loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # Save the model if this epoch has the best loss so far
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(autoencoder.state_dict(), 'best_autoencoder.pth')
        print(f"Saved better model with loss: {best_loss:.4f}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x30 and 2x64)

In [ ]:
# Initialize the model (ensure the model class is defined in your script)
autoencoder = TimeSeriesAutoencoder()

# Load the state dict
autoencoder.load_state_dict(torch.load('best_autoencoder.pth'))

# If you saved the entire model, you can directly load it (not recommended for large models)
# autoencoder = torch.load('full_autoencoder.pth')

In [ ]:
for i, (inputs, _) in enumerate(data_loader):
    print(inputs[1])

In [ ]:
# After training your autoencoder
# Example for 50 points of data
num_points = 50

# Loop through the data loader for 50 points
for i, (inputs, _) in enumerate(data_loader):
    if i >= num_points:
        break
    # Get the encoded and decoded output
    encoded, decoded = autoencoder(inputs)

    # Inverse normalize the decoded output
    decoded_np = decoded.detach().numpy() 

    # Compare with original data
    original_np = inputs.detach().numpy()

    # Visualization for Price
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)
    plt.plot(original_np[:, 0], label='Original Price')
    plt.plot(decoded_np[:, 0], label='Decoded Price')
    plt.title(f'Price Time Series - Original vs. Decoded (Point {i + 1})')
    plt.legend()

    # Visualization for Volume
    plt.subplot(2, 1, 2)
    plt.plot(original_np[:, 1], label='Original Volume')
    plt.plot(decoded_np[:, 1], label='Decoded Volume')
    plt.title(f'Volume Time Series - Original vs. Decoded (Point {i + 1})')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# Assuming you already have a batch of data
inputs, _ = next(iter(data_loader))
inputs = inputs.transpose(1, 2)  # Transposing to [batch_size, 2, 90]

# Get the encoded output
encoded, _ = autoencoder(inputs)

# Shapes
original_shape = inputs[0,:,:].shape
encoded_shape = encoded[0,:,:].shape

# Sizes (total number of elements)
original_size = inputs[0,:,:].nelement()
encoded_size = encoded[0,:,:].nelement()

# Printing shapes and sizes
print(f"Original Shape: {original_shape}, Size: {original_size}")
print(f"Encoded Shape: {encoded_shape}, Size: {encoded_size}")

# Calculating and printing reduction
reduction = (1 - (encoded_size / original_size)) * 100
print(f"Reduction in size: {reduction:.2f}%")
